In [66]:
import pandas as pd
import yfinance as yf
import requests
import re
import json
import plotly.express as px



urls = {
    "Nasdaq": 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nasdaq/nasdaq_full_tickers.json',
    "NYSE": 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nyse/nyse_full_tickers.json',
    "AMEX": 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/amex/amex_full_tickers.json'
}

# Now urls is a dictionary with the keys "Nasdaq", "NYSE", and "AMEX" each associated with their respective URL
print(urls)

# Fetch the content of the JSON file
df = pd.DataFrame()

for key, url in urls.items():
    response = requests.get(url)
    df_tmp = pd.DataFrame(response.json())
    df_tmp['Exchange'] = key
    df = pd.concat([df, df_tmp], ignore_index = True)

# save a local copy
# df.to_csv('us_symbol_tickers.csv', index = False)
df.head()

{'Nasdaq': 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nasdaq/nasdaq_full_tickers.json', 'NYSE': 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/nyse/nyse_full_tickers.json', 'AMEX': 'https://raw.githubusercontent.com/rreichel3/US-Stock-Symbols/main/amex/amex_full_tickers.json'}


,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url,Exchange
0,AACG,ATA Creativity Global American Depositary Shares,$0.8178,0.0159,1.983%,11820,26169706.00,China,2008,Other Consumer Services,Real Estate,/market-activity/stocks/aacg,Nasdaq
1,AACI,Armada Acquisition Corp. I Common Stock,$11.44,-0.01,-0.087%,4196,0.00,United States,2021,Blank Checks,Finance,/market-activity/stocks/aaci,Nasdaq
2,AACIU,Armada Acquisition Corp. I Unit,$11.43,0.00,0.00%,17,0.00,United States,2021,Blank Checks,Finance,/market-activity/stocks/aaciu,Nasdaq
3,AADI,Aadi Bioscience Inc. Common Stock,$1.46,-0.17,-10.429%,2405439,35849139.00,United States,,Biotechnology: Pharmaceutical Preparations,Health Care,/market-activity/stocks/aadi,Nasdaq
4,AAGR,African Agriculture Holdings Inc. Common Stock,$0.2697,0.0281,11.631%,59802194,15606684.00,United States,2021,Farming/Seeds/Milling,Consumer Staples,/market-activity/stocks/aagr,Nasdaq


In [67]:

df['marketCap'] = pd.to_numeric(df['marketCap'], errors='coerce')
df['marketCap'] = df['marketCap'].fillna(0)
df['marketCap'] = df['marketCap'].astype(int)

df = df.sort_values(by=['marketCap'], ascending= False).reset_index(drop = True)
df['MarketCap_pct'] = df['marketCap'] / df['marketCap'].sum()
df['MarketCap_pct_cumsum'] = df['MarketCap_pct'].cumsum()

# Fill empty values in 'industry' and 'sector' columns with 'Other'
df['industry'].replace('', 'Other', inplace=True)
df['sector'].replace('', 'Other', inplace=True)

len(df['symbol'].str.strip().drop_duplicates())

7106

In [68]:
# df[df['MarketCap_pct_cumsum'] <=0.]

top_100_stocks = df.head(100)

top_100_stocks_list =top_100_stocks['symbol'].to_list()

# Group by industry and select top 10 stocks per industry based on market capitalization
top_10_stocks_per_industry = df.groupby('industry').apply(lambda x: x.nlargest(10, 'marketCap')).reset_index(drop=True)

top_10_stocks_per_sector = df.groupby('sector').apply(lambda x: x.nlargest(10, 'marketCap')).reset_index(drop=True)

top_10_stocks_per_sector

,symbol,name,lastsale,netchange,pctchange,volume,marketCap,country,ipoyear,industry,sector,url,Exchange,MarketCap_pct,MarketCap_pct_cumsum
0,RIO,Rio Tinto Plc Common Stock,$65.93,0.11,0.167%,1756276,107275251010,Australia,2002,Metal Mining,Basic Materials,/market-activity/stocks/rio,NYSE,0.001451,0.590671
1,SCCO,Southern Copper Corporation Common Stock,$107.74,0.01,0.009%,916904,83295525274,United States,,Metal Mining,Basic Materials,/market-activity/stocks/scco,NYSE,0.001126,0.638405
2,FCX,Freeport-McMoRan Inc. Common Stock,$48.60,0.28,0.579%,12545504,69813412882,United States,,Metal Mining,Basic Materials,/market-activity/stocks/fcx,NYSE,0.000944,0.667875
3,VALE,VALE S.A. American Depositary Shares Each Rep...,$11.17,0.01,0.09%,24790386,50700714535,Brazil,,Metal Mining,Basic Materials,/market-activity/stocks/vale,NYSE,0.000686,0.719028
4,NEM,Newmont Corporation,$41.87,0.03,0.072%,10265529,48281979965,United States,,Precious Metals,Basic Materials,/market-activity/stocks/nem,NYSE,0.000653,0.727053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,RSG,Republic Services Inc. Common Stock,$194.34,-0.48,-0.246%,4550515,61203974082,United States,1998,Environmental Services,Utilities,/market-activity/stocks/rsg,NYSE,0.000828,0.689782
126,ET,Energy Transfer LP Common Units,$16.22,0.19,1.185%,12582238,54661778218,United States,,Natural Gas Distribution,Utilities,/market-activity/stocks/et,NYSE,0.000739,0.705524
127,WMB,Williams Companies Inc. (The) Common Stock,$42.50,0.26,0.616%,8506822,51797045808,United States,,Natural Gas Distribution,Utilities,/market-activity/stocks/wmb,NYSE,0.000700,0.716265
128,PCG,Pacific Gas & Electric Co. Common Stock,$17.46,-0.02,-0.114%,24117604,50272173015,United States,,Power Generation,Utilities,/market-activity/stocks/pcg,NYSE,0.000680,0.721076


In [69]:
top_100_stocks = df.head(100)
top_100_stocks_list =top_100_stocks['symbol'].to_list()
top_100_stocks_list.remove('BRK/A')
top_100_stocks_list.remove('BRK/B')

In [70]:
# yf.download('BRK', start = '2018-1-1')

In [71]:
stock_price = yf.download(top_100_stocks_list, start = '2018-1-1')

[*********************100%%**********************]  98 of 98 completed


In [72]:
# Flatten the MultiIndex columns
stock_price.columns = ['_'.join(col) for col in stock_price.columns]

# Convert to long format
long_format_df = pd.wide_to_long(stock_price.reset_index(), stubnames=['Price', 'Adj Close', 'Close', 'High', 'Low', 'Open','Volume'], i='Date', j='Company', sep='_', suffix='\\w+')

# Reset index to get the desired DataFrame format
long_format_df = long_format_df.reset_index()

long_format_df.drop(columns='Price', inplace = True)

long_format_df

,Date,Company,Adj Close,Close,High,Low,Open,Volume
0,2018-01-02,AAPL,40.615883,43.064999,43.075001,42.314999,42.540001,102223600.0
1,2018-01-03,AAPL,40.608807,43.057499,43.637501,42.990002,43.132500,118071600.0
2,2018-01-04,AAPL,40.797443,43.257500,43.367500,43.020000,43.134998,89738400.0
3,2018-01-05,AAPL,41.261932,43.750000,43.842499,43.262501,43.360001,94640000.0
4,2018-01-08,AAPL,41.108677,43.587502,43.902500,43.482498,43.587502,82271200.0
...,...,...,...,...,...,...,...,...
160029,2024-06-24,XOM,114.050003,114.050003,114.529999,111.639999,111.639999,16266700.0
160030,2024-06-25,XOM,114.370003,114.370003,114.550003,113.459999,114.110001,16213700.0
160031,2024-06-26,XOM,114.410004,114.410004,114.820000,113.139999,114.389999,15771800.0
160032,2024-06-27,XOM,114.900002,114.900002,114.989998,114.110001,114.989998,16812700.0


In [81]:
long_format_df.merge(df, left_on = 'Company', right_on = 'symbol', how = 'left')

,Date,Company,Adj Close,Close,High,Low,Open,Volume,symbol,name,...,volume,marketCap,country,ipoyear,industry,sector,url,Exchange,MarketCap_pct,MarketCap_pct_cumsum
0,2018-01-02,AAPL,40.615883,43.064999,43.075001,42.314999,42.540001,102223600.0,AAPL,Apple Inc. Common Stock,...,82475486,3229664350840,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl,Nasdaq,0.043671,0.088588
1,2018-01-03,AAPL,40.608807,43.057499,43.637501,42.990002,43.132500,118071600.0,AAPL,Apple Inc. Common Stock,...,82475486,3229664350840,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl,Nasdaq,0.043671,0.088588
2,2018-01-04,AAPL,40.797443,43.257500,43.367500,43.020000,43.134998,89738400.0,AAPL,Apple Inc. Common Stock,...,82475486,3229664350840,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl,Nasdaq,0.043671,0.088588
3,2018-01-05,AAPL,41.261932,43.750000,43.842499,43.262501,43.360001,94640000.0,AAPL,Apple Inc. Common Stock,...,82475486,3229664350840,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl,Nasdaq,0.043671,0.088588
4,2018-01-08,AAPL,41.108677,43.587502,43.902500,43.482498,43.587502,82271200.0,AAPL,Apple Inc. Common Stock,...,82475486,3229664350840,United States,1980,Computer Manufacturing,Technology,/market-activity/stocks/aapl,Nasdaq,0.043671,0.088588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160029,2024-06-24,XOM,114.050003,114.050003,114.529999,111.639999,111.639999,16266700.0,XOM,Exxon Mobil Corporation Common Stock,...,24564757,453918950414,United States,,Integrated oil Companies,Energy,/market-activity/stocks/xom,NYSE,0.006138,0.344828
160030,2024-06-25,XOM,114.370003,114.370003,114.550003,113.459999,114.110001,16213700.0,XOM,Exxon Mobil Corporation Common Stock,...,24564757,453918950414,United States,,Integrated oil Companies,Energy,/market-activity/stocks/xom,NYSE,0.006138,0.344828
160031,2024-06-26,XOM,114.410004,114.410004,114.820000,113.139999,114.389999,15771800.0,XOM,Exxon Mobil Corporation Common Stock,...,24564757,453918950414,United States,,Integrated oil Companies,Energy,/market-activity/stocks/xom,NYSE,0.006138,0.344828
160032,2024-06-27,XOM,114.900002,114.900002,114.989998,114.110001,114.989998,16812700.0,XOM,Exxon Mobil Corporation Common Stock,...,24564757,453918950414,United States,,Integrated oil Companies,Energy,/market-activity/stocks/xom,NYSE,0.006138,0.344828


In [73]:
# df = yf.download(['WMT','AAPL'], start = '2024-6-28') 

In [74]:
# index_Dow_list = ['AMZN','AXP','AMGN','AAPL','BA','CAT','CSCO','CVX','GS','HD','HON','IBM','INTC','JNJ','KO','JPM','MCD','MMM','MRK','MSFT','NKE','PG','TRV','UNH','CRM','VZ','V','WMT','DIS','DOW']

In [75]:
# def get_gdp(freq, series_id): 

#     # Define the URL and parameters
#     url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'

#     param = [('dataset', 'IFS'),
#             ('freq', freq),
#             ('country', 'US'),
#             ('series', series_id), 
#             ('start', '?startPeriod=1920')]


#     series = '.'.join([i[1] for i in param[1:4]])

#     key = f'CompactData/{param[0][1]}/{series}{param[-1][1]}'

#     # Combine API url with key specific to data request
#     r = requests.get(f'{url}{key}').json()

#     # Extract the data portion of results
#     data = r['CompactData']['DataSet']['Series']

#     # Check if data is a list or a single dictionary
#     if isinstance(data, dict):
#         data = [data]

#     # Create pandas dataframe, column = country, row = obs
#     df = pd.DataFrame({s['@REF_AREA']:
#                     {i['@TIME_PERIOD']: float(i['@OBS_VALUE']) 
#                         for i in s['Obs']} for s in data})

#     # Convert index to datetime
#     df.index = pd.to_datetime(df.index)
    
#     return df

1.2 CPI and PPI

https://data.bls.gov/cgi-bin/surveymost?bls


In [76]:
# series_id = ['WPSFD4'] #'CUUR0000SA0', 'WPSFD4','EIUIR','EIUIQ','LNS14000000' # PPI, CPI, Import, Export, Unemployment
start_year = '1920'
end_year = '2024'

def get_bls_data(series_id, start_year, end_year): 
    
    headers = {'Content-type': 'application/json'}
    
    series_data = json.dumps({"seriesid": [series_id] ,"startyear" : start_year, "endyear" : end_year})  

    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data = series_data, headers = headers)
    json_data = json.loads(p.text)
    for series in json_data['Results']['series']:
        x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
        seriesId = series['seriesID']
        for item in series['data']:
            year = item['year']
            period = item['period']
            value = item['value']
            footnotes=""
            for footnote in item['footnotes']:
                if footnote:
                    footnotes = footnotes + footnote['text'] + ','
            if 'M01' <= period <= 'M12':
                x.add_row([seriesId,year,period,value,footnotes[0:-1]])
        output = open(seriesId + '.txt','w')
        output.write (x.get_string())
        output.close()
        
    # Extract the relevant data
    series_data = json_data['Results']['series'][0]['data']
    # Convert to DataFrame

    df = pd.DataFrame(series_data)

    return df
    

In [77]:
# # CPI
# df_cpi = get_bls_data('WPSFD4', start_year, end_year)

# # PPI
# df_ppi = get_bls_data('CUUR0000SA0', start_year, end_year)

# # unemployment rate
# df_unemployment_rate = get_bls_data('LNS14000000', start_year, end_year) 

In [78]:
# df_ppi

In [79]:
# series_id = 'NGDP_XDC' # Nominal GDP in national currency 
# freq = 'A'

# df_gdp = get_gdp(freq, series_id)

In [80]:
#  for knowledge pages

    # df = pd.DataFrame({
    #     "Index": ['SP500', 'DJIA', 'NASDAQ100'],
    #     "Composition": ["500 Companies", "30 Blue-chip Companies", "100 Companies"],
    #     "Market_Value": ['70-80% US Market','25%',' '],
    #     "Method": ['Market Value Weighted Avg','Price Weighted Average','']
    # })

    # true_html = '<input type="checkbox" checked disabled="true">'
    # false_html = '<input type="checkbox" disabled="true">'

    # # df['D'] = df['D'].apply(lambda b: true_html if b else false_html)

    # st.markdown(df.to_html(escape=False), unsafe_allow_html=True)